In [ ]:
# Seasonal Decomposition

result = seasonal_decompose(df['Ped South'],model='addictive', period = 365)
result.plot()
plt.show()

In [ ]:
# # First, inspect the spike by visualizing it
# df['Ped South'].loc['2018-01-01':'2018-12-31'].plot()

# # If it's an anomaly (after investigation), you can either:
# # 1. Correct the values (set to NaN or replace with nearby values)
# df.loc['2018-01-01':'2018-12-31', 'Ped South'] = np.nan  # Replace with NaN
# # or 
# # 2. Remove the period (drop the rows)A
# df = df[df.index.year != 2018]  # Remove 2018 data entirely


In [ ]:
# df[['Ped North Z', 'Ped South Z', 'Bike North Z', 'Bike South Z']] = df[['Ped North', 'Ped South', 'Bike North', 'Bike South']].apply(zscore)
# plt.figure(figsize=(12,6))
# plt.plot(df.index, df['Ped North Z'],label = 'Ped North Z', color = 'green')
# plt.plot(df.index, df['Ped South Z'],label = 'Ped South Z', color = 'red')
# plt.plot(df.index, df['Bike North Z'],label = 'Bike North Z', color = 'yellow')
# plt.plot(df.index, df['Bike South Z'],label = 'Bike South Z', color = 'blue')
# plt.axhline(y= 3, color = 'red', linestyle = '--', label = 'Threshold')
# plt.axhline(y=3, color = 'red', linestyle = '--')
# plt.title('Z-score of pedestrian traffic (ped south) over time')
# plt.xlabel('Date')
# plt.ylabel('z-score')
# plt.legend()
# plt.xticks(rotation = 45)
# plt.show()

# anomalies = df[df['Ped North Z'].abs()>5]
# print(anomalies)

In [1]:
#SMOTE process
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# Separate features and target
X = df.drop("target_3hr", axis=1)
y = df["target_3hr"]


# model = XGBClassifier(scale_pos_weight=100, use_label_encoder=False, eval_metric="logloss")
# Impute missing values with the mean (or use "median", "most_frequent", etc.)
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_imputed, y)

# Combine back into DataFrame
resampled_df = pd.concat([
    pd.DataFrame(X_resampled, columns=X.columns),
    pd.Series(y_resampled, name="target_3hr")
], axis=1)

NameError: name 'df' is not defined

In [ ]:
# Balanced dataset 
# Step 1: Separate majority and minority classes
majority_class = df[df["target_3hr"] == 0]
minority_class = df[df["target_3hr"] == 1]

# Step 2: Sample only if population is large enough
n_samples = min(len(majority_class), len(minority_class))
majority_class_sampled = majority_class.sample(n=n_samples, random_state=42)
minority_class_sampled = minority_class.sample(n=n_samples, random_state=42)

# Step 3: Combine both
balanced_df = pd.concat([majority_class_sampled, minority_class_sampled])

# Step 4: Shuffle
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# features =  ['lag_1hr',	'lag_2hr','lag_3hr']
# Step 3: Split into Features and Target
X = balanced_df.drop("target_3hr", axis=1)
y = balanced_df["target_3hr"]

#split the dataset into training and test dataset

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 42) 
model = RandomForestClassifier(class_weight='balanced',n_estimators=50,random_state= 42)
# model = XGBClassifier(scale_pos_weight=100, use_label_encoder=False, eval_metric="logloss")

model.fit(X_train,Y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model


# Get prediction probabilities
y_probs = model.predict_proba(X_test)[:, 1]  # Get probability of class 1

# Adjust threshold to increase recall
threshold = 0.3  # Lower threshold to catch more anomalies
y_pred_adjusted = (y_probs >= threshold).astype(int)

accuracy = accuracy_score(Y_test, y_pred_adjusted)
print(f"Model Accuracy: {accuracy:.2f}")
print(classification_report(Y_test, y_pred))

In [ ]:

# df['residual'] = df['actual'] - df['predicted']

# # Flag residuals as anomalies if they exceed 3 standard deviations
# df['anomaly'] = df['residual'].apply(lambda x: 1 if abs(x) > 3 * df['residual'].std() else 0)


#### remaining features (rolling mean 3h, rolling std 3h,rolling 7d avg,month)

In [ ]:
# from typing import Union
# from fastapi import FastAPI

# app = FastAPI()

# # @app.get("/")
# def root():
#     print("hello this is inferenece model")

# @app.get("/data")
# def get_data():
    

